In [1]:
%matplotlib inline
from netpyne import specs, sim
import matplotlib.pyplot as plt
import pandas as pd
from tqdm.notebook import tqdm
import numpy as np

In [6]:
# Network parameters
netParams = specs.NetParams()  # object of class NetParams to store the network parameters


## Cell parameters
netParams.cellParams['pyr'] = {
    'secs': {
        'soma': {
            'geom': {
                'diam': 18.8,   
                'L': 18.8, 
                'Ra': 123.0},
            'mechs': {
                'hh': {
                    'gnabar': 0.12, 
                    'gkbar': 0.036, 
                    'gl': 0.0003, 
                    'el': -70}
            },
            'ions':{
                'k': {
                    'e': -85,
                }
                
            }
        }
    }
}                


## Population parameters
netParams.popParams['E'] = {
    'cellType': 'pyr', 
    'numCells': 40}

netParams.popParams['I'] = {
    'cellType': 'pyr', 
    'numCells': 10}


## Synaptic mechanism parameters
netParams.synMechParams['exc'] = {
    'mod': 'Exp2Syn', 
    'tau1': 0.1, 
    'tau2': 5.0, 
    'e': 0}  # excitatory synaptic mechanism

netParams.synMechParams['inh'] = {
    'mod': 'Exp2Syn', 
    'tau1': 0.1, 
    'tau2': 5.0, 
    'e': -70}  # inhibitory synaptic mechanism


# Stimulation parameters
netParams.stimSourceParams['bkg'] = {
    'type': 'NetStim', 
    'rate': 50, 
    'noise': 0.5}
    
netParams.stimTargetParams['bkg->E'] = {
    'source': 'bkg', 
    'conds': {'pop': 'E'}, 
    'weight': 0.01, 
    'delay': 5, 
    'synMech': 'exc'}


## Connectivity rules
netParams.connParams['E->I'] = {    #  S -> I label
    'preConds': {'pop': 'E'},       # conditions of presyn cells
    'postConds': {'pop': 'I'},      # conditions of postsyn cells
    'divergence': 5,               # probability of connection
    'weight': 0.01,                 # synaptic weight
    'delay': 5,                     # transmission delay (ms)
    'synMech': 'exc'}               # synaptic mechanism

netParams.connParams['I->E'] = {    #  I -> S label
    'preConds': {'pop': 'I'},       # conditions of presyn cells
    'postConds': {'pop': 'E'},      # conditions of postsyn cells
    'probability': 0.7,               # probability of connection
    'weight': 0.02,                 # synaptic weight
    'delay': 5,                     # transmission delay (ms)
    'synMech': 'inh'}               # synaptic mechanism


# Simulation options
simConfig = specs.SimConfig()       # object of class SimConfig to store simulation configuration

simConfig.duration = 25*1e3          # Duration of the simulation, in ms
simConfig.dt = 0.01                # Internal integration timestep to use
simConfig.verbose = False           # Show detailed messages
simConfig.recordTraces = {'V_soma':{'sec':'soma','loc':0.5,'var':'v'}}  # Dict with traces to record
simConfig.recordStep = 5        # Step size in ms to save data (eg. V traces, LFP, etc)
simConfig.filename = 'tut_oscillation'  # Set file output name
simConfig.saveJson = False

simConfig.recordLFP = [[50, 50, 50]]
simConfig.recordDipole = True

simConfig.analysis['plotTraces'] = {'include': [1], 'saveFig': False}  # Plot recorded traces for this list of cells
simConfig.analysis['plotRaster'] = {'showFig': False}                  # Plot a raster
simConfig.analysis['plotSpikeHist'] = {'include': ['E', 'I'], 'showFig': False}
simConfig.analysis['plotRateSpectrogram'] = {'include': ['all']}


# Create network and run simulation
# sim.createSimulateAnalyze(netParams = netParams, simConfig = simConfig)

In [7]:
qtd_sample = 50
all_eeg = []
targets = []
for param_k in tqdm([-55, -85]):
    #netParams.cellParams['pyr']["secs"]["soma"]["ions"]["k"]["e"] = param_k
    for j in tqdm(range(qtd_sample)):
        netParams.cellParams['pyr']["secs"]["soma"]["ions"]["k"]["e"] = param_k + np.random.randn()*0.1
        netParams.stimSourceParams['bkg']['rate'] = 50+np.random.randn()*0.1
        netParams.stimSourceParams['bkg']['noise'] = 0.5+np.random.randn()*0.01
        sim.createSimulateAnalyze(netParams = netParams, simConfig = simConfig)
        array, closest_elec_idx = sim.analysis.plotEEG()
        save_array = array[closest_elec_idx,400:]
        all_eeg.append(save_array)
        targets.append(param_k)

  0%|          | 0/2 [00:00<?, ?it/s]


Start time:  2023-01-18 23:29:20.045607

Creating network of 2 cell populations on 1 hosts...
  Number of cells on node 0: 50 
  Done; cell creation time = 0.00 s.
Making connections...
  Number of connections on node 0: 475 
  Done; cell connection time = 0.03 s.
Adding stims...
  Number of stims on node 0: 40 
  Done; cell stims creation time = 0.00 s.
Recording 1 traces of 1 types on node 0

Running simulation using NEURON for 25000.0 ms...
  Done; run time = 219.31 s; real-time ratio: 0.11.

Gathering data...
  Done; gather time = 0.01 s.

Analyzing...
  Cells: 50
  Connections: 515 (10.30 per cell)
  Spikes: 89 (0.07 Hz)
  Simulated time: 25.0 s; 1 workers
  Run time: 219.31 s
Closest electrode to dipole: 16.76 mm
38

Start time:  2023-01-18 23:33:01.740365

Creating network of 2 cell populations on 1 hosts...
  Number of cells on node 0: 50 
  Done; cell creation time = 0.01 s.
Making connections...
  Number of connections on node 0: 475 
  Done; cell connection time = 0.07 s.
A

  Done; run time = 215.50 s; real-time ratio: 0.12.

Gathering data...
  Done; gather time = 0.01 s.

Analyzing...
  Cells: 50
  Connections: 515 (10.30 per cell)
  Spikes: 89 (0.07 Hz)
  Simulated time: 25.0 s; 1 workers
  Run time: 215.50 s
Closest electrode to dipole: 16.76 mm
38

Start time:  2023-01-19 00:13:16.369293

Creating network of 2 cell populations on 1 hosts...
  Number of cells on node 0: 50 
  Done; cell creation time = 0.01 s.
Making connections...
  Number of connections on node 0: 475 
  Done; cell connection time = 0.06 s.
Adding stims...
  Number of stims on node 0: 40 
  Done; cell stims creation time = 0.01 s.
Recording 1 traces of 1 types on node 0

Running simulation using NEURON for 25000.0 ms...
  Done; run time = 215.82 s; real-time ratio: 0.12.

Gathering data...
  Done; gather time = 0.01 s.

Analyzing...
  Cells: 50
  Connections: 515 (10.30 per cell)
  Spikes: 89 (0.07 Hz)
  Simulated time: 25.0 s; 1 workers
  Run time: 215.82 s
Closest electrode to dip

Closest electrode to dipole: 16.76 mm
38

Start time:  2023-01-19 00:53:15.140258

Creating network of 2 cell populations on 1 hosts...
  Number of cells on node 0: 50 
  Done; cell creation time = 0.01 s.
Making connections...
  Number of connections on node 0: 475 
  Done; cell connection time = 0.06 s.
Adding stims...
  Number of stims on node 0: 40 
  Done; cell stims creation time = 0.00 s.
Recording 1 traces of 1 types on node 0

Running simulation using NEURON for 25000.0 ms...
  Done; run time = 215.59 s; real-time ratio: 0.12.

Gathering data...
  Done; gather time = 0.01 s.

Analyzing...
  Cells: 50
  Connections: 515 (10.30 per cell)
  Spikes: 88 (0.07 Hz)
  Simulated time: 25.0 s; 1 workers
  Run time: 215.59 s
Closest electrode to dipole: 16.76 mm
38

Start time:  2023-01-19 00:56:52.976139

Creating network of 2 cell populations on 1 hosts...
  Number of cells on node 0: 50 
  Done; cell creation time = 0.01 s.
Making connections...
  Number of connections on node 0: 475 

  Done; run time = 215.81 s; real-time ratio: 0.12.

Gathering data...
  Done; gather time = 0.01 s.

Analyzing...
  Cells: 50
  Connections: 515 (10.30 per cell)
  Spikes: 88 (0.07 Hz)
  Simulated time: 25.0 s; 1 workers
  Run time: 215.81 s
Closest electrode to dipole: 16.76 mm
38

Start time:  2023-01-19 01:36:51.526861

Creating network of 2 cell populations on 1 hosts...
  Number of cells on node 0: 50 
  Done; cell creation time = 0.01 s.
Making connections...
  Number of connections on node 0: 475 
  Done; cell connection time = 0.06 s.
Adding stims...
  Number of stims on node 0: 40 
  Done; cell stims creation time = 0.00 s.
Recording 1 traces of 1 types on node 0

Running simulation using NEURON for 25000.0 ms...
  Done; run time = 215.51 s; real-time ratio: 0.12.

Gathering data...
  Done; gather time = 0.01 s.

Analyzing...
  Cells: 50
  Connections: 515 (10.30 per cell)
  Spikes: 89 (0.07 Hz)
  Simulated time: 25.0 s; 1 workers
  Run time: 215.51 s
Closest electrode to dip

Closest electrode to dipole: 16.76 mm
38

Start time:  2023-01-19 02:16:47.361703

Creating network of 2 cell populations on 1 hosts...
  Number of cells on node 0: 50 
  Done; cell creation time = 0.01 s.
Making connections...
  Number of connections on node 0: 475 
  Done; cell connection time = 0.06 s.
Adding stims...
  Number of stims on node 0: 40 
  Done; cell stims creation time = 0.01 s.
Recording 1 traces of 1 types on node 0

Running simulation using NEURON for 25000.0 ms...
  Done; run time = 216.04 s; real-time ratio: 0.12.

Gathering data...
  Done; gather time = 0.01 s.

Analyzing...
  Cells: 50
  Connections: 515 (10.30 per cell)
  Spikes: 89 (0.07 Hz)
  Simulated time: 25.0 s; 1 workers
  Run time: 216.04 s
Closest electrode to dipole: 16.76 mm
38

Start time:  2023-01-19 02:20:25.639301

Creating network of 2 cell populations on 1 hosts...
  Number of cells on node 0: 50 
  Done; cell creation time = 0.01 s.
Making connections...
  Number of connections on node 0: 475 

  Done; run time = 140.78 s; real-time ratio: 0.18.

Gathering data...
  Done; gather time = 0.01 s.

Analyzing...
  Cells: 50
  Connections: 515 (10.30 per cell)
  Spikes: 49946 (39.96 Hz)
  Simulated time: 25.0 s; 1 workers
  Run time: 140.78 s
Closest electrode to dipole: 16.76 mm
38

Start time:  2023-01-19 02:50:17.782819

Creating network of 2 cell populations on 1 hosts...
  Number of cells on node 0: 50 
  Done; cell creation time = 0.01 s.
Making connections...
  Number of connections on node 0: 475 
  Done; cell connection time = 0.06 s.
Adding stims...
  Number of stims on node 0: 40 
  Done; cell stims creation time = 0.00 s.
Recording 1 traces of 1 types on node 0

Running simulation using NEURON for 25000.0 ms...
  Done; run time = 139.67 s; real-time ratio: 0.18.

Gathering data...
  Done; gather time = 0.01 s.

Analyzing...
  Cells: 50
  Connections: 515 (10.30 per cell)
  Spikes: 50029 (40.02 Hz)
  Simulated time: 25.0 s; 1 workers
  Run time: 139.67 s
Closest electrod

Closest electrode to dipole: 16.76 mm
38

Start time:  2023-01-19 03:16:18.258726

Creating network of 2 cell populations on 1 hosts...
  Number of cells on node 0: 50 
  Done; cell creation time = 0.01 s.
Making connections...
  Number of connections on node 0: 475 
  Done; cell connection time = 0.06 s.
Adding stims...
  Number of stims on node 0: 40 
  Done; cell stims creation time = 0.01 s.
Recording 1 traces of 1 types on node 0

Running simulation using NEURON for 25000.0 ms...
  Done; run time = 138.48 s; real-time ratio: 0.18.

Gathering data...
  Done; gather time = 0.01 s.

Analyzing...
  Cells: 50
  Connections: 515 (10.30 per cell)
  Spikes: 50001 (40.00 Hz)
  Simulated time: 25.0 s; 1 workers
  Run time: 138.48 s
Closest electrode to dipole: 16.76 mm
38

Start time:  2023-01-19 03:18:38.981910

Creating network of 2 cell populations on 1 hosts...
  Number of cells on node 0: 50 
  Done; cell creation time = 0.01 s.
Making connections...
  Number of connections on node 0: 

  Done; run time = 138.16 s; real-time ratio: 0.18.

Gathering data...
  Done; gather time = 0.01 s.

Analyzing...
  Cells: 50
  Connections: 515 (10.30 per cell)
  Spikes: 49945 (39.96 Hz)
  Simulated time: 25.0 s; 1 workers
  Run time: 138.16 s
Closest electrode to dipole: 16.76 mm
38

Start time:  2023-01-19 03:44:39.932342

Creating network of 2 cell populations on 1 hosts...
  Number of cells on node 0: 50 
  Done; cell creation time = 0.01 s.
Making connections...
  Number of connections on node 0: 475 
  Done; cell connection time = 0.06 s.
Adding stims...
  Number of stims on node 0: 40 
  Done; cell stims creation time = 0.00 s.
Recording 1 traces of 1 types on node 0

Running simulation using NEURON for 25000.0 ms...
  Done; run time = 139.51 s; real-time ratio: 0.18.

Gathering data...
  Done; gather time = 0.01 s.

Analyzing...
  Cells: 50
  Connections: 515 (10.30 per cell)
  Spikes: 49838 (39.87 Hz)
  Simulated time: 25.0 s; 1 workers
  Run time: 139.51 s
Closest electrod

Closest electrode to dipole: 16.76 mm
38

Start time:  2023-01-19 04:10:48.797232

Creating network of 2 cell populations on 1 hosts...
  Number of cells on node 0: 50 
  Done; cell creation time = 0.01 s.
Making connections...
  Number of connections on node 0: 475 
  Done; cell connection time = 0.06 s.
Adding stims...
  Number of stims on node 0: 40 
  Done; cell stims creation time = 0.01 s.
Recording 1 traces of 1 types on node 0

Running simulation using NEURON for 25000.0 ms...
  Done; run time = 145.90 s; real-time ratio: 0.17.

Gathering data...
  Done; gather time = 0.02 s.

Analyzing...
  Cells: 50
  Connections: 515 (10.30 per cell)
  Spikes: 49955 (39.96 Hz)
  Simulated time: 25.0 s; 1 workers
  Run time: 145.90 s
Closest electrode to dipole: 16.76 mm
38

Start time:  2023-01-19 04:13:17.022322

Creating network of 2 cell populations on 1 hosts...
  Number of cells on node 0: 50 
  Done; cell creation time = 0.01 s.
Making connections...
  Number of connections on node 0: 

In [8]:
df = pd.DataFrame(all_eeg)

In [9]:
df["target"]=targets

In [10]:
df.to_csv("eeg_dataset.csv")

## Split data

In [32]:
file_path = "eeg_dataset.csv"
df = pd.read_csv(file_path)

In [33]:
def split_df(df):
    
    new_healthy_data = []
    new_healthy_targets = []
    
    new_not_healthy_data = []
    new_not_healthy_targets = []
    
    df.drop(['Unnamed: 0'], inplace=True, axis=1)
    
    df_healthy = df.iloc[:50]
    df_not_healthy = df.iloc[50:]
    
    df_healthy.drop(["target"], axis=1, inplace=True)
    df_not_healthy.drop(["target"], axis=1, inplace=True)
    
    for i in df_healthy.to_numpy():
        new_healthy_data+=np.split(i, 23)
    healthy_targets = ["-55" for i in range(len(new_healthy_data))]

    
    for i in df_not_healthy.to_numpy():
        new_not_healthy_data+=np.split(i, 23)
    not_healthy_targets = ["-85" for i in range(len(new_not_healthy_data))]
        
    data = new_healthy_data+new_not_healthy_data
    target = healthy_targets+not_healthy_targets
    
    final_df = pd.DataFrame(data)
    final_df["target"] = target
        
    return final_df

In [34]:
final_df = split_df(df)

In [35]:
final_df.to_csv("eeg_dataset_splited_3.csv")

## merge data

In [43]:
pd.set_option('display.max_rows', None)

In [36]:
df_1 = pd.read_csv("eeg_dataset_splited_1.csv")
df_2 = pd.read_csv("eeg_dataset_splited_2.csv")
df_3 = pd.read_csv("eeg_dataset_splited_3.csv")

In [51]:
df_dataset = pd.concat([df_1, df_2, df_3])
df_dataset.sort_values(["target"], inplace=True)

In [52]:
df_dataset.to_csv("eeg_netpyne_dataset.csv")

## Split train and test data

In [55]:
df = pd.read_csv("dataset/eeg_netpyne_dataset.csv")

msk = np.random.rand(len(df)) < 0.8

train = df[msk]

test = df[~msk]

In [58]:
train.target.value_counts()

-55    2761
-85    2042
Name: target, dtype: int64

In [59]:
test.target.value_counts()

-55    689
-85    488
Name: target, dtype: int64

In [60]:
train.to_csv("dataset/train_eeg_netpyne_dataset.csv")
test.to_csv("dataset/test_eeg_netpyne_dataset.csv")